In [4]:
from IngestDrugSynonyms import IngestDrugSynonyms

In [5]:
import pandas as pd

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
drugSynonym = IngestDrugSynonyms()

In [11]:
drugSynonym.auto_get_and_clean_data()

2020-04-10 10:57:54,880 - > STARTING scraping with 'covid-19' keyword
2020-04-10 10:57:58,961 - > 427 studies found by 'covid-19' keyword
2020-04-10 10:58:12,226 - > STARTING scraping with 'SARS-CoV-2' keyword
2020-04-10 10:58:15,421 - > 427 studies found by 'SARS-CoV-2' keyword
2020-04-10 10:58:27,698 - > STARTING scraping with 'coronavirus' keyword
2020-04-10 10:58:31,700 - > 346 studies found by 'coronavirus' keyword
2020-04-10 10:58:48,359 - > 1076 distinct studies found


In [14]:
drug_vocab = drugSynonym.drug_vocab
drugs_and_syms:list = list(drug_vocab.keys())

drugs_and_syms.extend( item for key in drug_vocab.keys() if isinstance(drug_vocab[key],list) for item in drug_vocab[key])

In [15]:
len(drugs_and_syms)

39481

In [ ]:
ids_and_inte

In [16]:
import time

In [ ]:
studies = drugSynonym.all_studies_df
appeared_in_edges:list = []
start = time.time()
print(start)
count = 0
prev = 0
for drug in drugs_and_syms:
    
    for index,row in studies.iterrows():
        if drug in row["intervention"]:
            appeared_in_edges.append((drug,row["trial_id"]))
    count += 1
    if count > prev + 1000:
        print(time.time())
        print(time.time() - start)
        prev = count
            
end = time.time()
print(end - start)
print(end)


In [ ]:
start = time.time()
appeared_in_edges:list = [(drug,row["trial_id"]) for drug in drugs_and_syms for index,row in studies.iterrows() if drug in row["intervention"]]
end = time.time()
print(end - start)

In [ ]:
39481 * 83

In [ ]:
import cudf